In [ ]:
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe
from google.oauth2.service_account import Credentials
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

In [ ]:
import gspread
from google.oauth2.service_account import Credentials
import pandas as pd
import pandas as pd
import json
import io
import time
import pandas
from pandas import json_normalize
import requests
import ast
from datetime import datetime
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe
from google.oauth2.service_account import Credentials
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
startTime = datetime.now()

def fetch_order(order_id):
    try:
        response = requests.get(f"https://api-b2b.lamoda.ru/api/v1/orders/{order_id}", headers=headers)
        if response.status_code == 200:
            return response.json()  # Return JSON response
        else:
            return {"error": f"Status {response.status_code}: {response.text}"}
    except Exception as e:
        return {"error": str(e)}




# 1. Получаем токен
auth_url = "https://api-b2b.lamoda.ru/auth/token"
auth_payload = {
    "client_id": "752_65npbusbnqwwg4cw8s0k0wkcsg44888socc800wskkssssowk8",
    "client_secret": "1qfa6xvrvrq8gkgwksk0wgccwwo0s0sssc04o8c0wo0koc44w8",
    "grant_type": "client_credentials"
}
headers = {"Content-Type": "application/x-www-form-urlencoded"}

auth_response = requests.post(auth_url, data=auth_payload, headers=headers)

if auth_response.status_code == 200:
    access_token = auth_response.json().get("access_token")
    print("Токен получен:", access_token)
else:
    print("Ошибка получения токена:", auth_response.text)
    exit()

page = 1
# page_size = 500  # Adjust based on API limits
all_orders = []

url = 'https://api-b2b.lamoda.ru/api/v1/orders'
api_key = access_token
while True:
    print(f"Fetching page {page}...")
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    params = {
        "page": page,

        "limit": 3000}

    # params = {"page": page, "limit": page_size}
    response = requests.get(url, headers=headers, params=params)
    result = response.json()

    all_orders.extend(result['_embedded']['orders'])

    if page == int(result["pages"]):
        break

    page += 1  # Move to next page
    time.sleep(1)  # Add delay to avoid rate limits
print("вышли из цикла")
df = pd.json_normalize(all_orders)

delivered_data = df.loc[df['status'] == 'Delivered']



headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

expanded_list = []

order_details = delivered_data["id"].apply(fetch_order)

expanded_list.extend(order_details)

df_expand = pd.json_normalize(expanded_list)


list_data = []
for index, row in df_expand.iterrows():
    total_sales = row['_embedded.items']
    try:
        #print(total_sales)
        for i in range(len(total_sales)):
            list_data.append(total_sales[i])
            #print(total_sales[i])
    except:
        print("Fail"+ total_sales)


df_final = pd.DataFrame(list_data)
finaldf = df_final.loc[df_final['status'] == 'Delivered']

df_expand['id'] = df_expand['id'].str.replace("C","")

merged_df = pd.merge(finaldf, df_expand, on='id', how='outer')

finaldf = merged_df[['id', 'sku','basePrice_x','salePrice_x','paidPrice_x','partnerAgreedPrice_x','status_x','createdAt','dates.delivered']]

finaldf = finaldf.ffill()
# order_details = df_final["id"].apply(fetch_order)
# print('order_details')
# expanded_list.extend(order_details)


# df_expand = pd.json_normalize(expanded_list)
#result_df = pd.merge(delivered_data, df_expand, on="id")

# result_df['_embedded.items'] = result_df['_embedded.items'].apply(ast.literal_eval)
# result_df['sku_column2'] = result_df['_embedded.items'].apply(
#     lambda items: items[0]['sku'] if items and isinstance(items[0], dict) and 'sku' in items[0] else None
# )

#finaldf = df_expand[['id', 'sku','basePrice_x','salePrice_x','paidPrice_x','partnerAgreedPrice_x','status_x','createdAt','dates.delivered']]

#finaldf = finaldf.loc[finaldf['status_x'] == 'Delivered']

# finaldf = finaldf.ffill()

finaldf.to_csv('Lamoda_data.csv')
print(datetime.now() - startTime)

# Укажите путь к вашему файлу credentials.json
SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']

creds = Credentials.from_service_account_file('/content/halogen-pier-451511-d0-7b522bfe360b.json', scopes=SCOPES)
client = gspread.authorize(creds)

# Создать новую таблицу
#spreadsheet = client.create("Новая таблица")

# Или открыть существующую таблицу по имени
spreadsheet = client.open("Игорь / аналитика амазонка")

# Выберите первый лист (или любой другой по имени) https://docs.google.com/spreadsheets/d/1ApnRhKoWfwe66idRprHOulvC3QwDmoakWZ_B_TTmnhQ/edit?gid=496060326#gid=496060326
#sheet = spreadsheet.worksheet('496060326')
# sheet_gid = "496060326"  # Например, "0" для первого листа

# # Найти лист по gid
# sheet = None
# for worksheet in spreadsheet.worksheets():
#     if str(worksheet.id) == sheet_gid:
#         sheet = worksheet
#         break

# Преобразуем DataFrame в список списков
#values = [finaldf.columns.values.tolist()] + finaldf.values.tolist()
worksheet1 = spreadsheet.worksheet("Продажи Lamoda")

worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe=finaldf, include_index=False,
include_column_header=True, resize=True)
# # Вставляем данные в Google Sheets
# sheet.update('A1', values)
# #sheet.update(range_name='A1', values=values)

print("Данные успешно вставлены в Google Sheets!")




Токен получен: YmViNTgxODFiZjFlNjA5MzAxMDRmOTI1MDMyZDgxNmQ2NjI2ZThmODU4NGM0MzUxNDJkOTdiNjI3ZDEwZmM2Zg
Fetching page 1...
Fetching page 2...
Fetching page 3...
Fetching page 4...
Fetching page 5...
Fetching page 6...
Fetching page 7...
Fetching page 8...
Fetching page 9...
Fetching page 10...
Fetching page 11...
Fetching page 12...
вышли из цикла
2:12:37.248630


In [ ]:
# # Укажите путь к вашему файлу credentials.json
# SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']

# creds = Credentials.from_service_account_file('/content/halogen-pier-451511-d0-7b522bfe360b.json', scopes=SCOPES)
# client = gspread.authorize(creds)

# # Создать новую таблицу
# #spreadsheet = client.create("Новая таблица")

# # Или открыть существующую таблицу по имени
# spreadsheet = client.open("Игорь / аналитика амазонка")

# # Выберите первый лист (или любой другой по имени) https://docs.google.com/spreadsheets/d/1ApnRhKoWfwe66idRprHOulvC3QwDmoakWZ_B_TTmnhQ/edit?gid=496060326#gid=496060326
# #sheet = spreadsheet.worksheet('496060326')
# # sheet_gid = "496060326"  # Например, "0" для первого листа

# # # Найти лист по gid
# # sheet = None
# # for worksheet in spreadsheet.worksheets():
# #     if str(worksheet.id) == sheet_gid:
# #         sheet = worksheet
# #         break

# # Преобразуем DataFrame в список списков
# #values = [finaldf.columns.values.tolist()] + finaldf.values.tolist()
# worksheet1 = spreadsheet.worksheet("Продажи Lamoda")

# worksheet1.clear()
# set_with_dataframe(worksheet=worksheet1, dataframe=finaldf, include_index=False,
# include_column_header=True, resize=True)
# # # Вставляем данные в Google Sheets
# # sheet.update('A1', values)
# # #sheet.update(range_name='A1', values=values)

# print("Данные успешно вставлены в Google Sheets!")


In [ ]:
# worksheet1.clear()
# set_with_dataframe(worksheet=worksheet1, dataframe=finaldf, include_index=False,
# include_column_header=True, resize=True)
# # # Вставляем данные в Google Sheets
# # sheet.update('A1', values)
# # #sheet.update(range_name='A1', values=values)

# print("Данные успешно вставлены в Google Sheets!")

In [ ]:
df

In [ ]:
# worksheet1.clear()
# set_with_dataframe(worksheet=worksheet1, dataframe=finaldf, include_index=False,
# include_column_header=True, resize=True)
# # # Вставляем данные в Google Sheets
# # sheet.update('A1', values)
# # #sheet.update(range_name='A1', values=values)

# print("Данные успешно вставлены в Google Sheets!")

In [ ]:
import pandas as pd
import json
import io
import time
import pandas
from pandas import json_normalize
import requests
import ast
from datetime import datetime
startTime = datetime.now()

In [ ]:
auth_url = "https://api-b2b.lamoda.ru/auth/token"
auth_payload = {
    "client_id": "752_65npbusbnqwwg4cw8s0k0wkcsg44888socc800wskkssssowk8",
    "client_secret": "1qfa6xvrvrq8gkgwksk0wgccwwo0s0sssc04o8c0wo0koc44w8",
    "grant_type": "client_credentials"
}
headers = {"Content-Type": "application/x-www-form-urlencoded"}

auth_response = requests.post(auth_url, data=auth_payload, headers=headers)

if auth_response.status_code == 200:
    access_token = auth_response.json().get("access_token")
    print("Токен получен:", access_token)
else:
    print("Ошибка получения токена:", auth_response.text)
    exit()



Токен получен: MTFlMzFkZjk1MGZkNjgwZGM3Y2JiMWI0NTFmYzVmYjdiN2E5NjkzZDBhNmJiMTY4MzFiZGJjMzlkMmI3OWU5Mw


In [ ]:
url = 'https://api-b2b.lamoda.ru/api/v1/stock/goods'
api_key = access_token
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}
params = {
   # "page": page,

    "limit": 100}

# params = {"page": page, "limit": page_size}
response = requests.get(url, headers=headers, params=params)
result = response.json()

In [ ]:
GET /api/v1/stock/goods?sku[]={id}

In [ ]:
https://b2b-guide.lamoda.ru/specification#operation/get_v1-stock-goods

{'page': 1,
 'limit': 100,
 'pages': 19,
 'total': 1845,
 '_links': {'self': {'href': '/api/v1/stock/goods?page=1&limit=100'},
  'first': {'href': '/api/v1/stock/goods?page=1&limit=100'},
  'last': {'href': '/api/v1/stock/goods?page=19&limit=100'},
  'next': {'href': '/api/v1/stock/goods?page=2&limit=100'}},
 '_embedded': {'stockStates': [{'sku': 'LaMo_bust_Clarimond/black 95C',
    'quantity': 2},
   {'sku': 'LaMo_bust_Emlyn/black 5XL', 'quantity': 0},
   {'sku': 'LaMo_bust_Clarimond/black 80D', 'quantity': 3},
   {'sku': 'LaMo_bust_Clarimond/black 85C', 'quantity': 4},
   {'sku': 'LaMo_bust_Emlyn/black 4XL', 'quantity': 0},
   {'sku': 'LaMo_bust_Emlyn/black M', 'quantity': 0},
   {'sku': 'LaMo_bust_Clarimond/black 75B', 'quantity': 1},
   {'sku': 'LaMo_bust_Emlyn/black XXL', 'quantity': 0},
   {'sku': 'LaMo_bust_Clarimond/black 75C', 'quantity': 2},
   {'sku': 'LaMo_bust_Clarimond/black 75A', 'quantity': 2},
   {'sku': 'LaMo_bust_Clarimond/black 90D', 'quantity': 3},
   {'sku': 'LaMo

In [ ]:
import gspread
from google.oauth2.service_account import Credentials
import pandas as pd
import pandas as pd
import json
import io
import time
import pandas
from pandas import json_normalize
import requests
import ast
from datetime import datetime
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe
from google.oauth2.service_account import Credentials
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
startTime = datetime.now()
page = 1
# page_size = 500  # Adjust based on API limits
all_stocks = []

url = 'https://api-b2b.lamoda.ru/api/v1/stock/goods'
api_key = access_token
while True:
    print(f"Fetching page {page}...")
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    params = {
        "page": page,
      #  "withZeroQuantity":0,

        "limit": 500}


    response = requests.get(url, headers=headers, params=params)
    result = response.json()
    #print(result['_embedded'])
    all_stocks.extend(result['_embedded']['stockStates'])

    if page == int(result['pages']):
        break

    page += 1  # Move to next page
    time.sleep(1)  # Add delay to avoid rate limits

df = pd.json_normalize(all_stocks)

# Укажите путь к вашему файлу credentials.json
SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']

creds = Credentials.from_service_account_file('/content/halogen-pier-451511-d0-7b522bfe360b.json', scopes=SCOPES)
client = gspread.authorize(creds)

# Создать новую таблицу
#spreadsheet = client.create("Новая таблица")

# Или открыть существующую таблицу по имени
spreadsheet = client.open("Игорь / аналитика амазонка")

# Выберите первый лист (или любой другой по имени) https://docs.google.com/spreadsheets/d/1ApnRhKoWfwe66idRprHOulvC3QwDmoakWZ_B_TTmnhQ/edit?gid=496060326#gid=496060326
#sheet = spreadsheet.worksheet('496060326')
# sheet_gid = "496060326"  # Например, "0" для первого листа

# # Найти лист по gid
# sheet = None
# for worksheet in spreadsheet.worksheets():
#     if str(worksheet.id) == sheet_gid:
#         sheet = worksheet
#         break

# Преобразуем DataFrame в список списков
#values = [finaldf.columns.values.tolist()] + finaldf.values.tolist()
worksheet1 = spreadsheet.worksheet("Остатки на Складе Lamoda")

worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe=df, include_index=False,
include_column_header=True, resize=True)
# # Вставляем данные в Google Sheets
# sheet.update('A1', values)
# #sheet.update(range_name='A1', values=values)

print("Данные успешно вставлены в Google Sheets!")


Fetching page 1...
Fetching page 2...
Fetching page 3...
Fetching page 4...


In [ ]:
# import pandas as pd
# import json
# import io
# import time
# import pandas
# from pandas import json_normalize
# import requests
# import ast
# from datetime import datetime
# startTime = datetime.now()

# def fetch_order(order_id):
#     try:
#         response = requests.get(f"https://api-b2b.lamoda.ru/api/v1/orders/{order_id}", headers=headers)
#         if response.status_code == 200:
#             return response.json()  # Return JSON response
#         else:
#             return {"error": f"Status {response.status_code}: {response.text}"}
#     except Exception as e:
#         return {"error": str(e)}




# # 1. Получаем токен
# auth_url = "https://api-b2b.lamoda.ru/auth/token"
# auth_payload = {
#     "client_id": "752_65npbusbnqwwg4cw8s0k0wkcsg44888socc800wskkssssowk8",
#     "client_secret": "1qfa6xvrvrq8gkgwksk0wgccwwo0s0sssc04o8c0wo0koc44w8",
#     "grant_type": "client_credentials"
# }
# headers = {"Content-Type": "application/x-www-form-urlencoded"}

# auth_response = requests.post(auth_url, data=auth_payload, headers=headers)

# if auth_response.status_code == 200:
#     access_token = auth_response.json().get("access_token")
#     print("Токен получен:", access_token)
# else:
#     print("Ошибка получения токена:", auth_response.text)
#     exit()

# page = 1
# # page_size = 500  # Adjust based on API limits
# all_orders = []

# url = 'https://api-b2b.lamoda.ru/api/v1/orders'
# api_key = access_token
# while True:
#     print(f"Fetching page {page}...")
#     headers = {
#         "Authorization": f"Bearer {api_key}",
#         "Content-Type": "application/json"
#     }
#     params = {
#         "page": page,

#         "limit": 3000}

#     # params = {"page": page, "limit": page_size}
#     response = requests.get(url, headers=headers, params=params)
#     result = response.json()

#     all_orders.extend(result['_embedded']['orders'])

#     if page == 3:# int(result["pages"]):
#         break

#     page += 1  # Move to next page
#     time.sleep(1)  # Add delay to avoid rate limits
# print("вышли из цикла")
# df = pd.json_normalize(all_orders)

# delivered_data = df.loc[df['status'] == 'Delivered']



# headers = {
#     "Authorization": f"Bearer {api_key}",
#     "Content-Type": "application/json"
# }

# expanded_list = []

# order_details = delivered_data["id"].apply(fetch_order)

# expanded_list.extend(order_details)

# df_expand = pd.json_normalize(expanded_list)


# list_data = []
# for index, row in df_expand.iterrows():
#     total_sales = row['_embedded.items']
#     try:
#         #print(total_sales)
#         for i in range(len(total_sales)):
#             list_data.append(total_sales[i])
#             print(total_sales[i])
#     except:
#         print("Fail"+ total_sales)


# df_final = pd.DataFrame(list_data)
# finaldf = df_final.loc[df_final['status'] == 'Delivered']

# df_expand['id'] = df_expand['id'].str.replace("C","")

# merged_df = pd.merge(finaldf, df_expand, on='id', how='outer')

# finaldf = merged_df[['id', 'sku','basePrice_x','salePrice_x','paidPrice_x','partnerAgreedPrice_x','status_x','createdAt','dates.delivered']]

# finaldf = finaldf.ffill()
# # order_details = df_final["id"].apply(fetch_order)
# # print('order_details')
# # expanded_list.extend(order_details)


# # df_expand = pd.json_normalize(expanded_list)
# #result_df = pd.merge(delivered_data, df_expand, on="id")

# # result_df['_embedded.items'] = result_df['_embedded.items'].apply(ast.literal_eval)
# # result_df['sku_column2'] = result_df['_embedded.items'].apply(
# #     lambda items: items[0]['sku'] if items and isinstance(items[0], dict) and 'sku' in items[0] else None
# # )

# #finaldf = df_expand[['id', 'sku','basePrice_x','salePrice_x','paidPrice_x','partnerAgreedPrice_x','status_x','createdAt','dates.delivered']]

# #finaldf = finaldf.loc[finaldf['status_x'] == 'Delivered']

# # finaldf = finaldf.ffill()

# finaldf.to_csv('Lamoda_data.csv')
# print(datetime.now() - startTime)






Output hidden; open in https://colab.research.google.com to view.

In [40]:
import pandas as pd
import json
import io
import time
import pandas
from pandas import json_normalize
import requests
import ast
from datetime import datetime

In [ ]:
# all_orders = []

# #url = 'https://api-b2b.lamoda.ru/api/v1/orders'
# #api_key = access_token
# while True:
#     print(f"Fetching page {page}...")
#     headers = {
#         "Authorization": f"Bearer {api_key}",
#         "Content-Type": "application/json"
#     }
#     params = {
#         "page": page,

#         "limit": 3000}

#     # params = {"page": page, "limit": page_size}
#     response = requests.get(url, headers=headers, params=params)
#     result = response.json()

#     all_orders.extend(result['_embedded']['orders'])

#     if page == 3:# int(result["pages"]):
#         break

#     page += 1  # Move to next page
#     time.sleep(1)  # Add delay to avoid rate limits
# print("вышли из цикла")
# df = pd.json_normalize(all_orders)

In [4]:
all_orders = []
offset = 0
while True:

    #url = 'https://api.moysklad.ru/api/remap/1.2/entity/product/metadata/attributes/<id>=12dd3ad1-bd03-11ed-0a80-04df008e3fb8'
    #url = 'https://api.moysklad.ru/api/remap/1.2/report/stock/all/current?stockType=quantity&include=zeroLines'
    #url = 'https://api.moysklad.ru/api/remap/1.2/report/stock/all'

    #url = f"https://api.moysklad.ru/api/remap/1.2/report/stock/all?limit=1000&offset={offset}"
    url = f"https://api.moysklad.ru/api/remap/1.2/report/stock/all?offset={offset}&limit=1000"
    api_key = '91747401152ee277431c2d47115df852fdb8a1a6'
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    response = requests.get(url, headers=headers) #, params=params)
    result = response.json()

    all_orders.extend(result['rows'])

    if offset >  int(result['meta']['size']):

        break
    print(offset)
    print(result['meta']['offset'])

    offset += 1000  # Move to next page
    time.sleep(5)  # Add delay to avoid rate limits
df = pd.json_normalize(all_orders)

0
0
1000
1000
2000
2000
3000
3000
4000
4000
5000
5000
6000
6000
7000
7000
8000
8000
9000
9000
10000
10000


In [8]:
with pd.ExcelWriter('moisklad.xlsx') as writer:
    # writing to the 'Employee' sheet
    df.to_excel(writer, sheet_name='moisklad', index=False)

In [42]:
url = f"https://api.moysklad.ru/api/remap/1.2/entity/assortment"
api_key = '91747401152ee277431c2d47115df852fdb8a1a6'
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

response = requests.get(url, headers=headers) #, params=params)
result = response.json()

In [43]:
result.keys()

In [39]:
result

{'meta': {'href': 'https://api.moysklad.ru/api/remap/1.2/entity/employee/e4dd23db-3e27-11ed-0a80-016600044318',
  'metadataHref': 'https://api.moysklad.ru/api/remap/1.2/entity/employee/metadata',
  'type': 'employee',
  'mediaType': 'application/json',
  'uuidHref': 'https://online.moysklad.ru/app/#employee/edit?id=e4dd23db-3e27-11ed-0a80-016600044318'},
 'id': 'e4dd23db-3e27-11ed-0a80-016600044318',
 'accountId': 'bd506798-03df-11eb-0a80-01d500013938',
 'owner': {'meta': {'href': 'https://api.moysklad.ru/api/remap/1.2/entity/employee/e4dd23db-3e27-11ed-0a80-016600044318',
   'metadataHref': 'https://api.moysklad.ru/api/remap/1.2/entity/employee/metadata',
   'type': 'employee',
   'mediaType': 'application/json',
   'uuidHref': 'https://online.moysklad.ru/app/#employee/edit?id=e4dd23db-3e27-11ed-0a80-016600044318'}},
 'shared': True,
 'group': {'meta': {'href': 'https://api.moysklad.ru/api/remap/1.2/entity/group/bd50c0f9-03df-11eb-0a80-01d500013939',
   'metadataHref': 'https://api.mo

In [37]:
result

{'meta': {'href': 'https://api.moysklad.ru/api/remap/1.2/entity/employee/e4dd23db-3e27-11ed-0a80-016600044318',
  'metadataHref': 'https://api.moysklad.ru/api/remap/1.2/entity/employee/metadata',
  'type': 'employee',
  'mediaType': 'application/json',
  'uuidHref': 'https://online.moysklad.ru/app/#employee/edit?id=e4dd23db-3e27-11ed-0a80-016600044318'},
 'id': 'e4dd23db-3e27-11ed-0a80-016600044318',
 'accountId': 'bd506798-03df-11eb-0a80-01d500013938',
 'owner': {'meta': {'href': 'https://api.moysklad.ru/api/remap/1.2/entity/employee/e4dd23db-3e27-11ed-0a80-016600044318',
   'metadataHref': 'https://api.moysklad.ru/api/remap/1.2/entity/employee/metadata',
   'type': 'employee',
   'mediaType': 'application/json',
   'uuidHref': 'https://online.moysklad.ru/app/#employee/edit?id=e4dd23db-3e27-11ed-0a80-016600044318'}},
 'shared': True,
 'group': {'meta': {'href': 'https://api.moysklad.ru/api/remap/1.2/entity/group/bd50c0f9-03df-11eb-0a80-01d500013939',
   'metadataHref': 'https://api.mo

In [35]:
result

{'meta': {'href': 'https://api.moysklad.ru/api/remap/1.2/entity/productfolder/75db881b-c36b-11ef-0a80-0f7c001c4b57',
  'metadataHref': 'https://api.moysklad.ru/api/remap/1.2/entity/productfolder/metadata',
  'type': 'productfolder',
  'mediaType': 'application/json',
  'uuidHref': 'https://online.moysklad.ru/app/#good/edit?id=75db881b-c36b-11ef-0a80-0f7c001c4b57'},
 'id': '75db881b-c36b-11ef-0a80-0f7c001c4b57',
 'accountId': 'bd506798-03df-11eb-0a80-01d500013938',
 'owner': {'meta': {'href': 'https://api.moysklad.ru/api/remap/1.2/entity/employee/e4dd23db-3e27-11ed-0a80-016600044318',
   'metadataHref': 'https://api.moysklad.ru/api/remap/1.2/entity/employee/metadata',
   'type': 'employee',
   'mediaType': 'application/json',
   'uuidHref': 'https://online.moysklad.ru/app/#employee/edit?id=e4dd23db-3e27-11ed-0a80-016600044318'}},
 'shared': True,
 'group': {'meta': {'href': 'https://api.moysklad.ru/api/remap/1.2/entity/group/d7f1c24e-9c7f-11eb-0a80-049300144009',
   'metadataHref': 'htt

In [26]:
result.keys()

dict_keys(['meta', 'id', 'accountId', 'owner', 'shared', 'group', 'updated', 'name', 'code', 'externalCode', 'archived', 'pathName', 'productFolder', 'useParentVat', 'uom', 'images', 'minPrice', 'salePrices', 'buyPrice', 'barcodes', 'attributes', 'paymentItemType', 'discountProhibited', 'article', 'weight', 'volume', 'variantsCount', 'isSerialTrackable', 'files', 'trackingType'])

In [27]:
df = pd.json_normalize(result)

In [31]:
df.columns

Index(['id', 'accountId', 'shared', 'updated', 'name', 'code', 'externalCode',
       'archived', 'pathName', 'useParentVat', 'salePrices', 'barcodes',
       'attributes', 'paymentItemType', 'discountProhibited', 'article',
       'weight', 'volume', 'variantsCount', 'isSerialTrackable',
       'trackingType', 'meta.href', 'meta.metadataHref', 'meta.type',
       'meta.mediaType', 'meta.uuidHref', 'owner.meta.href',
       'owner.meta.metadataHref', 'owner.meta.type', 'owner.meta.mediaType',
       'owner.meta.uuidHref', 'group.meta.href', 'group.meta.metadataHref',
       'group.meta.type', 'group.meta.mediaType', 'productFolder.meta.href',
       'productFolder.meta.metadataHref', 'productFolder.meta.type',
       'productFolder.meta.mediaType', 'productFolder.meta.uuidHref',
       'uom.meta.href', 'uom.meta.metadataHref', 'uom.meta.type',
       'uom.meta.mediaType', 'images.meta.href', 'images.meta.type',
       'images.meta.mediaType', 'images.meta.size', 'images.meta.limit',
  

In [9]:
result

{'meta': {'href': 'https://api.moysklad.ru/api/remap/1.2/entity/product/c39d54d6-f39a-11ec-0a80-0bcf000b1308?expand=supplier',
  'metadataHref': 'https://api.moysklad.ru/api/remap/1.2/entity/product/metadata',
  'type': 'product',
  'mediaType': 'application/json',
  'uuidHref': 'https://online.moysklad.ru/app/#good/edit?id=c39d4947-f39a-11ec-0a80-0bcf000b12f9'},
 'id': 'c39d54d6-f39a-11ec-0a80-0bcf000b1308',
 'accountId': 'bd506798-03df-11eb-0a80-01d500013938',
 'owner': {'meta': {'href': 'https://api.moysklad.ru/api/remap/1.2/entity/employee/39a8940a-0990-11ec-0a80-012d0033dfe9',
   'metadataHref': 'https://api.moysklad.ru/api/remap/1.2/entity/employee/metadata',
   'type': 'employee',
   'mediaType': 'application/json',
   'uuidHref': 'https://online.moysklad.ru/app/#employee/edit?id=39a8940a-0990-11ec-0a80-012d0033dfe9'}},
 'shared': True,
 'group': {'meta': {'href': 'https://api.moysklad.ru/api/remap/1.2/entity/group/d7f1c24e-9c7f-11eb-0a80-049300144009',
   'metadataHref': 'https

In [36]:
result['rows'][0]

{'meta': {'href': 'https://api.moysklad.ru/api/remap/1.2/entity/product/c39d54d6-f39a-11ec-0a80-0bcf000b1308?expand=supplier',
  'metadataHref': 'https://api.moysklad.ru/api/remap/1.2/entity/product/metadata',
  'type': 'product',
  'mediaType': 'application/json',
  'uuidHref': 'https://online.moysklad.ru/app/#good/edit?id=c39d4947-f39a-11ec-0a80-0bcf000b12f9'},
 'stock': 5.0,
 'inTransit': 0.0,
 'reserve': 0.0,
 'quantity': 5.0,
 'name': 'Тестовый товар на ИП Ермолин 25792041',
 'code': 'тестовый_код_1',
 'article': 'komplekt_Sofi_TEST/red L',
 'price': 46193.0,
 'salePrice': 0.0,
 'uom': {},
 'folder': {'meta': {'href': 'https://api.moysklad.ru/api/remap/1.2/entity/productfolder/1140c1b4-f397-11ec-0a80-04b80006acc1',
   'metadataHref': 'https://api.moysklad.ru/api/remap/1.2/entity/productfolder/metadata',
   'type': 'productfolder',
   'mediaType': 'application/json',
   'uuidHref': 'https://online.moysklad.ru/app/#good/edit?id=1140c1b4-f397-11ec-0a80-04b80006acc1'},
  'name': 'ИП Е

In [ ]:
result['rows'][0]

{'meta': {'href': 'https://api.moysklad.ru/api/remap/1.2/entity/product/c39d54d6-f39a-11ec-0a80-0bcf000b1308?expand=supplier',
  'metadataHref': 'https://api.moysklad.ru/api/remap/1.2/entity/product/metadata',
  'type': 'product',
  'mediaType': 'application/json',
  'uuidHref': 'https://online.moysklad.ru/app/#good/edit?id=c39d4947-f39a-11ec-0a80-0bcf000b12f9'},
 'stock': 5.0,
 'inTransit': 0.0,
 'reserve': 0.0,
 'quantity': 5.0,
 'name': 'Тестовый товар на ИП Ермолин 25792041',
 'code': 'тестовый_код_1',
 'article': 'komplekt_Sofi_TEST/red L',
 'price': 46193.0,
 'salePrice': 0.0,
 'uom': {},
 'folder': {'meta': {'href': 'https://api.moysklad.ru/api/remap/1.2/entity/productfolder/1140c1b4-f397-11ec-0a80-04b80006acc1',
   'metadataHref': 'https://api.moysklad.ru/api/remap/1.2/entity/productfolder/metadata',
   'type': 'productfolder',
   'mediaType': 'application/json',
   'uuidHref': 'https://online.moysklad.ru/app/#good/edit?id=1140c1b4-f397-11ec-0a80-04b80006acc1'},
  'name': 'ИП Е

In [ ]:
resulta = list(result.keys())
print(resulta)

AttributeError: 'list' object has no attribute 'keys'

In [ ]:
#url = 'https://api.moysklad.ru/api/remap/1.2/entity/metadata'
url = 'https://api.moysklad.ru/api/remap/1.2/entity/inventory/metadata'
api_key = '91747401152ee277431c2d47115df852fdb8a1a6'
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}
params = {
   # "page": page,

    "limit": 100}

# params = {"page": page, "limit": page_size}
response = requests.get(url, headers=headers) #, params=params)
result = response.json()